In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add
import cv2
import collections

Using TensorFlow backend.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
from Building_Vocab import load_data

### Load Data

In [4]:
li = load_data("./drive/My Drive/Files/variable_caption_processing.txt")
max_len = li[0]
vocab_size = li[1]

In [5]:
embedding_output = np.loadtxt("./drive/My Drive/Files/embedding_output.txt")

In [6]:
train_descriptions = load_data("./drive/My Drive/Files/train_descriptions.txt")

In [7]:
# Load the train images features from disk
with open("./drive/My Drive/Files/encoded_train_image_features.pkl", "rb") as encoded_pickle:
    encoding_train = pickle.load(encoded_pickle)

In [8]:
word_to_idx = load_data("./drive/My Drive/Files/word_to_idx.txt")

#### Model Architecture

In [18]:
# image feature extractor model
input_img_features = Input(shape=(2048,))
inp_img1 = Dropout(0.3)(input_img_features)
inp_img2 = Dense(256,activation='relu')(inp_img1)

In [19]:
# partial caption sequence model
input_captions = Input(shape=(max_len,))
inp_cap1 = Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_captions)
inp_cap2 = Dropout(0.3)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

In [20]:
# decoder (feed forward) model
decoder1 = add([inp_img2,inp_cap3])
decoder2 = Dense(256,activation='relu')(decoder1)
outputs = Dense(vocab_size,activation='softmax')(decoder2)

# Combined Model
model = Model(inputs=[input_img_features,input_captions],outputs=outputs)

In [21]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 35, 50)       92400       input_4[0][0]                    
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 2048)         0           input_3[0][0]                    
____________________________________________________________________________________________

In [22]:
model.layers[2].set_weights([embedding_output])
model.layers[2].trainable = False

In [23]:
model.compile(loss='categorical_crossentropy',optimizer="adam")

### Training of Model

#### Data Generator

In [24]:
def data_generator(train_descriptions, encoding_train, word_to_idx, max_len, num_photos_per_batch):

    X1, X2, y = [], [], []

    n=0

    while True:
        
        for key, desc_list in train_descriptions.items():
            n +=1

            photo = encoding_train[key+".jpg"]

            for desc in desc_list:
                
                seq = [ word_to_idx[word] for word in desc.split() if word in word_to_idx]  

                for i in range(1,len(seq)):

                    in_seq = seq[0:i]
                    out_seq = seq[i]

                    in_seq = pad_sequences([in_seq], maxlen=max_len, value=0, padding='post')[0]

                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                yield [[np.array(X1), np.array(X2)], np.array(y)]
                X1, X2, y = [], [], []
                n=0

In [25]:
epochs = 30
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [26]:
## training
for i in range(epochs):
    generator = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.h5')

Epoch 1/1
2000/2000 [==============================] - 306s 153ms/step - loss: 4.2877
Epoch 1/1
2000/2000 [==============================] - 308s 154ms/step - loss: 3.5727
Epoch 1/1
2000/2000 [==============================] - 306s 153ms/step - loss: 3.3191
Epoch 1/1
2000/2000 [==============================] - 307s 153ms/step - loss: 3.1658
Epoch 1/1
2000/2000 [==============================] - 308s 154ms/step - loss: 3.0511
Epoch 1/1
2000/2000 [==============================] - 308s 154ms/step - loss: 2.9681
Epoch 1/1
2000/2000 [==============================] - 306s 153ms/step - loss: 2.8952
Epoch 1/1
2000/2000 [==============================] - 306s 153ms/step - loss: 2.8408
Epoch 1/1
2000/2000 [==============================] - 305s 153ms/step - loss: 2.7914
Epoch 1/1
2000/2000 [==============================] - 307s 154ms/step - loss: 2.7520
Epoch 1/1
2000/2000 [==============================] - 307s 153ms/step - loss: 2.7164
Epoch 1/1
2000/2000 [==============================] -

In [27]:
epochs = 35
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [28]:
## training
for i in range(30,epochs):
    generator = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.h5')

Epoch 1/1
1000/1000 [==============================] - 178s 178ms/step - loss: 2.4065
Epoch 1/1
1000/1000 [==============================] - 176s 176ms/step - loss: 2.3824
Epoch 1/1
1000/1000 [==============================] - 178s 178ms/step - loss: 2.3710
Epoch 1/1
1000/1000 [==============================] - 176s 176ms/step - loss: 2.3624
Epoch 1/1
1000/1000 [==============================] - 177s 177ms/step - loss: 2.3554


In [29]:
epochs = 40
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [30]:
## training
for i in range(35,epochs):
    generator = data_generator(train_descriptions, encoding_train, word_to_idx, max_len, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.h5')

Epoch 1/1
1000/1000 [==============================] - 180s 180ms/step - loss: 2.3488
Epoch 1/1
1000/1000 [==============================] - 178s 178ms/step - loss: 2.3396
Epoch 1/1
1000/1000 [==============================] - 177s 177ms/step - loss: 2.3352
Epoch 1/1
1000/1000 [==============================] - 176s 176ms/step - loss: 2.3249
Epoch 1/1
1000/1000 [==============================] - 175s 175ms/step - loss: 2.3200


#### Downloading Models weights

In [32]:
!zip -r /model_weights.zip ./model_weights 

  adding: model_weights/ (stored 0%)
  adding: model_weights/model_12.h5 (deflated 16%)
  adding: model_weights/model_1.h5 (deflated 14%)
  adding: model_weights/model_29.h5 (deflated 16%)
  adding: model_weights/model_8.h5 (deflated 16%)
  adding: model_weights/model_21.h5 (deflated 16%)
  adding: model_weights/model_16.h5 (deflated 16%)
  adding: model_weights/model_17.h5 (deflated 16%)
  adding: model_weights/model_39.h5 (deflated 18%)
  adding: model_weights/model_32.h5 (deflated 16%)
  adding: model_weights/model_5.h5 (deflated 16%)
  adding: model_weights/model_24.h5 (deflated 16%)
  adding: model_weights/model_36.h5 (deflated 17%)
  adding: model_weights/model_15.h5 (deflated 16%)
  adding: model_weights/model_33.h5 (deflated 16%)
  adding: model_weights/model_11.h5 (deflated 16%)
  adding: model_weights/model_0.h5 (deflated 12%)
  adding: model_weights/model_25.h5 (deflated 16%)
  adding: model_weights/model_13.h5 (deflated 16%)
  adding: model_weights/model_31.h5 (deflated 16%

In [33]:
from google.colab import files
files.download("/model_weights.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>